In [2]:
import pandas as pd
import gzip
import nltk
from nltk.corpus import stopwords
import warnings; warnings.simplefilter('ignore')
import os, codecs, string, random
import numpy as np
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle
import matplotlib.pyplot as plt
%matplotlib inline  

seed = 42
random.seed(seed)
np.random.seed(seed)

#NLP libraries
import spacy, nltk, gensim, sklearn
#import pyLDAvis.gensim
import pyLDAvis.gensim_models

#Vader
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#Scikit imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# from textblob import TextBlob

In [3]:
# import small train and test datasets
train_neg = [tweet[:-1] for tweet in open('./twitter-datasets/train_neg.txt').readlines()]
train_pos = [tweet[:-1] for tweet in open('./twitter-datasets/train_pos.txt').readlines()]
test_data = [tweet[:-1] for tweet in open('./twitter-datasets/test_data.txt').readlines()]

# import vocab_cut
vocab = [word[:-1] for word in open('./vocab_cut.txt').readlines()]

In [4]:
count_vect = CountVectorizer(vocabulary = vocab)
X_train_counts = count_vect.fit_transform(train_neg)
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(100000, 101298)

## Pipeline using logistic regression

In [5]:
pipl = Pipeline([
    ('vect', CountVectorizer(vocabulary = vocab, ngram_range = (1, 5))),
    ('tfidf', TfidfTransformer()),
    ('logreg', LogisticRegression()),
])

In [6]:
X, y = train_neg + train_pos, [-1 for i in range(len(train_neg))]+[1 for i in range(len(train_pos))]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
pipl.fit(X_train, y_train)
prediction = pipl.predict(X_test)
print('accuracy = {}'.format(accuracy_score(y_test, prediction)))

accuracy = 0.81385


In [ ]:
# create a submission to AIcrowd
df = pd.DataFrame.from_dict({'Id' : range(1, 10001), 'Prediction': prediction.tolist()})
df.to_csv('submission1.csv', index=False)

## pipeline using decision tree classifier

In [7]:
pipl = Pipeline([
    ('vect', CountVectorizer(vocabulary = vocab, ngram_range = (1, 2))),
    ('clf', DecisionTreeClassifier(max_depth=50)),
])

In [8]:
X, y = train_neg + train_pos, [-1 for i in range(len(train_neg))]+[1 for i in range(len(train_pos))]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
pipl.fit(X_train, y_train)
prediction = pipl.predict(X_test)
print('accuracy = {}'.format(accuracy_score(y_test, prediction)))

accuracy = 0.7577


## Pipeline using neural networks

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
pipl = Pipeline([
    ('vect', CountVectorizer(ngram_range = (1, 1))),
    ('tfidf', TfidfTransformer()),
    ('mlp', MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 2), random_state=1)),
])

In [ ]:
# dont work
X, y = train_neg + train_pos, [-1 for i in range(len(train_neg))]+[1 for i in range(len(train_pos))]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipl.fit(X_train, y_train)
prediction = pipl.predict(X_test)
accuracy_score(y_test, prediction)